<a href="https://colab.research.google.com/github/Pavanishreebs/GuardianAI/blob/main/GuardianAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install psutil
!pip install requests

In [ ]:
import json
import socket
import urllib.request
import requests
import psutil  # For checking network connections

# Mock or replace with actual g4f implementation
def chat_with_gpt(prompt):
    # Replace this with actual g4f call if needed
    print(f"[GPT Prompt] {prompt}")
    return "[True, 85]"  # Mocked response

# Function to check if a URL is a phishing URL using GPT
def is_phishing_url(url):
    prompt = f"Only respond in this format [True/False, (if true : danger in % without % symbol. if False : None)] : Is this Url a Phishing url? {url}"
    response = chat_with_gpt(prompt)
    try:
        result = response.strip("[] ").split(",")
        is_phish = result[0].strip().lower() == 'true'
        danger = result[1].strip() if is_phish else None
        return is_phish, danger
    except Exception as e:
        print(f"Error parsing GPT response: {e}")
        return False, None

# Get IP address info using ip-api.com
def get_ip_address(url):
    try:
        response = requests.get(f'https://ip-api.com/json/{url}')
        response_json = response.json()
        return response_json.get('query', 'Unknown')
    except Exception as e:
        print(f"Failed to get IP for {url}: {e}")
        return "Unknown"

# Save to database.json
def save_phishing_url(url, ip_address):
    try:
        try:
            with open('database.json', 'r') as file:
                data = json.load(file)
        except (FileNotFoundError, json.JSONDecodeError):
            data = {}

        data[url] = ip_address

        with open('database.json', 'w') as file:
            json.dump(data, file, indent=4)
        print(f"Saved to database.json: {url} → {ip_address}")
    except Exception as e:
        print(f"Error saving URL to database: {e}")

# Get established network connections
def scan_connections():
    print("Scanning active connections...\n")
    for conn in psutil.net_connections(kind='inet'):
        if conn.status == 'ESTABLISHED' and conn.raddr:
            remote_ip = conn.raddr.ip
            try:
                # Try to resolve the IP to a URL (reverse DNS or http get)
                url = f"http://{remote_ip}"
                print(f"\nChecking: {url}")
                is_phish, danger = is_phishing_url(url)
                print(f"Phishing: {is_phish}, Risk: {danger}")

                if is_phish:
                    ip = get_ip_address(remote_ip)
                    save_phishing_url(url, ip)

            except Exception as e:
                print(f"Error checking connection to {remote_ip}: {e}")

# Run the scanner
if __name__ == "__main__":
    scan_connections()


Scanning active connections...


Checking: http://127.0.0.1
[GPT Prompt] Only respond in this format [True/False, (if true : danger in % without % symbol. if False : None)] : Is this Url a Phishing url? http://127.0.0.1
Phishing: True, Risk: 85
Saved to database.json: http://127.0.0.1 → Unknown

Checking: http://127.0.0.1
[GPT Prompt] Only respond in this format [True/False, (if true : danger in % without % symbol. if False : None)] : Is this Url a Phishing url? http://127.0.0.1
Phishing: True, Risk: 85
Saved to database.json: http://127.0.0.1 → Unknown

Checking: http://127.0.0.1
[GPT Prompt] Only respond in this format [True/False, (if true : danger in % without % symbol. if False : None)] : Is this Url a Phishing url? http://127.0.0.1
Phishing: True, Risk: 85
Saved to database.json: http://127.0.0.1 → Unknown

Checking: http://127.0.0.1
[GPT Prompt] Only respond in this format [True/False, (if true : danger in % without % symbol. if False : None)] : Is this Url a Phishing url? http: